In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

In [2]:
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive

# SD Card sector information
headerSector =          1022 # Holds user settings to configure Miniscope and recording
configSector =          1023 # Holds final settings of the actual recording
dataStartSector =       1024 # Recording data starts here
sectorSize =            512

WRITE_KEY0 =				0x0D7CBA17
WRITE_KEY1 =				0x0D7CBA17
WRITE_KEY2 =				0x0D7CBA17
WRITE_KEY3 =				0x0D7CBA17

# SD Card Header Sector positions
HEADER_GAIN_POS =				4
HEADER_LED_POS =				5
HEADER_EWL_POS =				6
HEADER_RECORD_LENGTH_POS =  	7
HEADER_FRAME_RATE = 			8

# SD Card Config Sector positions
CONFIG_BLOCK_WIDTH_POS =			    0
CONFIG_BLOCK_HEIGHT_POS	=   		    1
CONFIG_BLOCK_FRAME_RATE_POS	=   	    2
CONFIG_BLOCK_BUFFER_SIZE_POS =  	    3
CONFIG_BLOCK_NUM_BUFFERS_RECORDED_POS =	4
CONFIG_BLOCK_NUM_BUFFERS_DROPPED_POS =	5

# Data Buffer Header positions
BUFFER_HEADER_HEADER_LENGTH_POS =			0
BUFFER_HEADER_LINKED_LIST_POS = 			1
BUFFER_HEADER_FRAME_NUM_POS	=   			2
BUFFER_HEADER_BUFFER_COUNT_POS =			3
BUFFER_HEADER_FRAME_BUFFER_COUNT_POS =  	4
BUFFER_HEADER_WRITE_BUFFER_COUNT_POS =  	5
BUFFER_HEADER_DROPPED_BUFFER_COUNT_POS =	6
BUFFER_HEADER_TIMESTAMP_POS	=   			7
BUFFER_HEADER_DATA_LENGTH_POS = 			8
BUFFER_HEADER_WRITE_TIMESTAMP_POS = 		9

In [3]:
# Needs to be run as administrator to have access to openning and reading card

# variables that you can adjust
correctDrive = False
f = open(driveName, "rb")  # Open drive

# Make sure this is the correct drive
# Read SD Card header and config sectors
f.seek(headerSector * sectorSize, 0)  # Move to correct sector
headerSectorData = np.frombuffer(f.read(sectorSize), dtype=np.uint32)
if (True):
#if ((WRITE_KEY0 == headerSectorData[0]) and (WRITE_KEY1 == headerSectorData[1]) and (WRITE_KEY2 == headerSectorData[2]) and (WRITE_KEY3 == headerSectorData[3])):
    correctDrive = True
    print("SD Card Openned.")
else:
    print ("Wrong Drive.")
    correctDrive = False
    f.close()

SD Card Openned.


In [4]:
    # Do not normally need to run this cell
    
    #f.close()

In [5]:
# Load up Config Sector

f.seek(configSector * sectorSize, 0)  # Move to correct sector
configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)



# This needs to be removed once WF Miniscope writes this sector correctly
# configSectorData[CONFIG_BLOCK_WIDTH_POS] = 304
# configSectorData[CONFIG_BLOCK_HEIGHT_POS] = 304
# configSectorData[CONFIG_BLOCK_NUM_BUFFERS_RECORDED_POS] = 290
configSectorData

C:\Users\Federico\AppData\Local\Temp\ipykernel_20556\72588980.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)


array([   304,    304,      0,  20480, 128849,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [6]:
# Read Data Sectors
saveVideo = True
plotHeaderValues = False
displayVideo = True

frameNum = 0
pixelCount = 0
header = []

if saveVideo is True:
    out = cv2.VideoWriter('WFV4.avi', cv2.VideoWriter_fourcc(*'GREY'), 
                        10.0, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] ), 
                        isColor=False)

frame = np.zeros((configSectorData[CONFIG_BLOCK_WIDTH_POS] * configSectorData[CONFIG_BLOCK_HEIGHT_POS], 1), dtype=np.uint8)
f.seek(dataStartSector * sectorSize, 0) # Starting data location
for i in range(configSectorData[CONFIG_BLOCK_NUM_BUFFERS_RECORDED_POS]):
    dataHeader = np.fromstring(f.read(4), dtype=np.uint32) # gets header length
    dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))

    header.append(dataHeader)
    
    numBlocks = int((dataHeader[BUFFER_HEADER_DATA_LENGTH_POS] + (dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4) + (512 - 1)) / 512)
    
    data = np.fromstring(f.read(numBlocks*512 - dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4), dtype=np.uint8)

    # -------------------------------------
    if (dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] == 0):
        # First buffer of a frame
        
        if saveVideo is True:
            out.write(np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
        
        if displayVideo is True:
            cv2.imshow('Video', np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
            cv2.waitKey(20)
            
        frame[0:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS], 0] = data
        pixelCount = dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        frameNum = dataHeader[BUFFER_HEADER_FRAME_NUM_POS]
    else:
        # All other buffers of a frame
        # startIdx = dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] * 50 * 512
        frame[pixelCount:(pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]), 0] = data[:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]]
        pixelCount = pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]

if saveVideo is True:            
    out.release()

if displayVideo is True:
    cv2.destroyWindow('Video')
    
if plotHeaderValues is True:
    temp = np.asarray(header)
    plt.plot(temp)

C:\Users\Federico\AppData\Local\Temp\ipykernel_20556\2274884287.py:18: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dataHeader = np.fromstring(f.read(4), dtype=np.uint32) # gets header length
C:\Users\Federico\AppData\Local\Temp\ipykernel_20556\2274884287.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))
C:\Users\Federico\AppData\Local\Temp\ipykernel_20556\2274884287.py:25: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(f.read(numBlocks*512 - dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4), dtype=np.uint8)


In [7]:
header

[array([   10,     0,     0,     0,     0,     0,     0,   136, 20440,
          136], dtype=uint32),
 array([   10,     1,     0,     1,     1,     1,     0,   141, 20440,
          141], dtype=uint32),
 array([   10,     2,     0,     2,     2,     2,     0,   146, 20440,
          146], dtype=uint32),
 array([   10,     3,     0,     3,     3,     3,     0,   151, 20440,
          153], dtype=uint32),
 array([   10,     4,     0,     4,     4,     4,     0,   154, 10656,
          156], dtype=uint32),
 array([   10,     5,     1,     5,     0,     5,     0,   187, 20440,
          187], dtype=uint32),
 array([   10,     6,     1,     6,     1,     6,     0,   192, 20440,
          192], dtype=uint32),
 array([   10,     7,     1,     7,     2,     7,     0,   197, 20440,
          197], dtype=uint32),
 array([   10,     0,     1,     8,     3,     8,     0,   202, 20440,
          205], dtype=uint32),
 array([   10,     1,     1,     9,     4,     9,     0,   205, 10656,
          2